# 🔍 ChromaDB Viewer

Notebook per visualizzare e esplorare il contenuto del database ChromaDB.

In [ ]:
import sys
from pathlib import Path

# Aggiungi source al path per importare i moduli
sys.path.insert(0, str(Path("../source").resolve()))

from AI_RM import EmbeddingStore, Embedder, default_config
print("✅ Moduli caricati")

## 1. Connessione al Database

In [ ]:
# Inizializza lo store
store = EmbeddingStore()

# Mostra statistiche
stats = store.get_stats()
print(f"📊 Database Statistics")
print(f"   Collection: {stats.collection_name}")
print(f"   Documents: {stats.total_documents}")
print(f"   Path: {stats.persist_path}")
print(f"   Metric: {stats.distance_metric}")

## 2. Visualizza Sample di Video

In [ ]:
# Mostra alcuni video casuali
samples = store.sample(n=5)

if samples:
    print("🎬 Sample Videos:")
    for i, s in enumerate(samples, 1):
        print(f"\n{i}. {s['video_id']}")
        print(f"   Text: {s['text']}")
        print(f"   Category: {s['metadata'].get('category', 'N/A')}")
        print(f"   Channel: {s['metadata'].get('channel_name', 'N/A')}")
else:
    print("⚠️ Database vuoto! Esegui prima il notebook 02_chromadb_setup.ipynb")

## 3. Esporta come DataFrame

In [ ]:
# Esporta tutto il database in un DataFrame
df = store.to_dataframe()

if not df.empty:
    print(f"📋 DataFrame: {len(df)} righe, {len(df.columns)} colonne")
    print(f"\nColonne: {list(df.columns)}")
    display(df.head(10))
else:
    print("⚠️ Database vuoto!")

## 4. Distribuzione per Categoria

In [ ]:
import matplotlib.pyplot as plt

categories = store.get_category_distribution()

if categories:
    plt.figure(figsize=(10, 6))
    plt.barh(list(categories.keys()), list(categories.values()))
    plt.xlabel('Numero Video')
    plt.ylabel('Categoria')
    plt.title('Distribuzione Video per Categoria')
    plt.tight_layout()
    plt.show()
    
    print("\n📊 Dettaglio:")
    for cat, count in sorted(categories.items(), key=lambda x: -x[1]):
        print(f"   {cat}: {count}")
else:
    print("⚠️ Nessuna categoria trovata")

## 5. Distribuzione per Canale

In [ ]:
channels = store.get_channel_distribution()

if channels:
    # Top 15 canali
    top_channels = dict(sorted(channels.items(), key=lambda x: -x[1])[:15])
    
    plt.figure(figsize=(10, 6))
    plt.barh(list(top_channels.keys()), list(top_channels.values()))
    plt.xlabel('Numero Video')
    plt.ylabel('Canale')
    plt.title('Top 15 Canali per Numero Video')
    plt.tight_layout()
    plt.show()
else:
    print("⚠️ Nessun canale trovato")

## 6. Test Ricerca Semantica

In [ ]:
# Inizializza embedder per le query
embedder = Embedder()

def search(query: str, n: int = 5, category: str = None):
    """Funzione helper per cercare video."""
    query_emb = embedder.encode(query)
    where_filter = {"category": category} if category else None
    
    results = store.search(
        query_embedding=query_emb,
        n_results=n,
        where=where_filter
    )
    
    print(f"🔍 Query: '{query}'")
    if category:
        print(f"   Filter: category = '{category}'")
    print(f"\n📺 Risultati ({len(results)}):")
    
    for i, r in enumerate(results, 1):
        print(f"\n{i}. [{r.score:.3f}] {r.text[:70]}...")
        print(f"   ID: {r.video_id}")
        print(f"   Channel: {r.metadata.get('channel_name', 'N/A')}")
        print(f"   Category: {r.metadata.get('category', 'N/A')}")

In [ ]:
# Esempio di ricerca
search("python programming tutorial", n=5)

In [ ]:
# Ricerca con filtro categoria
search("machine learning", n=3, category="Education")

## 7. Lookup Video Specifico

In [ ]:
# Cerca un video specifico per ID
# Sostituisci con un ID reale dal tuo database

video_id = df['video_id'].iloc[0] if not df.empty else None

if video_id:
    video = store.get_video(video_id)
    if video:
        print(f"🎬 Video: {video_id}")
        print(f"\nText: {video['text'][:200]}...")
        print(f"\nMetadata: {video['metadata']}")
        print(f"\nEmbedding shape: {video['embedding'].shape if video['embedding'] is not None else 'N/A'}")
else:
    print("⚠️ Nessun video nel database")